In [2]:
import pandas as pd

df = pd.read_csv('/content/100_Unique_QA_Dataset.csv')
df.head()

,question,answer
0,What is the capital of France?,Paris
1,What is the capital of Germany?,Berlin
2,Who wrote 'To Kill a Mockingbird'?,Harper-Lee
3,What is the largest planet in our solar system?,Jupiter
4,What is the boiling point of water in Celsius?,100


In [12]:
#tokenize
def tokenize(text):
  text = text.lower()
  text = text.replace("?", "")
  text = text.replace("'", "")
  return text.split()


In [13]:
tokenize("What is the capital of France?")

['what', 'is', 'the', 'capital', 'of', 'france']

In [18]:
#form vocabulary
vocab = {'<UNK>':0 }
for question, answer in zip(df['question'], df['answer']):
  for token in (tokenize(question)+tokenize(answer)):
    if token not in vocab:
      vocab[token] = len(vocab)

print(len(vocab))

324


In [40]:
# convert words to numerical indices
def text_to_indices(text, vocab):

  indexed_text = []

  for token in tokenize(text):

    if token in vocab:
      indexed_text.append(vocab[token])
    else:
      indexed_text.append(vocab['<UNK>'])

  return indexed_text

In [41]:
text_to_indices("What is campusx", vocab)

[1, 2, 0]

In [42]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn

In [43]:
class QADataset(Dataset):

  def __init__(self, df, vocab):
    self.df = df
    self.vocab = vocab

  def __len__(self):
    return self.df.shape[0]

  def __getitem__(self, index):

    numerical_question = text_to_indices(self.df.iloc[index]['question'], self.vocab)
    numerical_answer = text_to_indices(self.df.iloc[index]['answer'], self.vocab)

    return torch.tensor(numerical_question), torch.tensor(numerical_answer)

In [44]:
dataset = QADataset(df, vocab)

In [45]:
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

In [ ]:
for question, answer in dataloader:
  print(question, answer[0])

RNN architecture

In [62]:
class simpleRNN(nn.Module):
  def __init__(self, vocab_size):
    super().__init__()
    self.embedding = nn.Embedding(vocab_size, embedding_dim=50)
    self.rnn = nn.RNN(50, 64, batch_first = True)
    self.fc = nn.Linear(64, vocab_size)

  def forward(self, question):
    embedded_question = self.embedding(question)
    hidden, final = self.rnn(embedded_question)
    output = self.fc(final.squeeze(0))
    return output

In [63]:
learning_rate = 0.001
epochs = 20

In [64]:
model = simpleRNN(len(vocab))
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [67]:
#training loop
for epoch in range(epochs):
  total_loss = 0
  for question, answer in dataloader:
    optimizer.zero_grad()
    output = model(question)
    loss = criterion(output, answer[0])
    loss.backward()
    optimizer.step()
    total_loss += loss.item()
  print(f"Epoch: {epoch+1}, Loss: {total_loss/len(dataloader)}")

Epoch: 1, Loss: 5.848601791593763
Epoch: 2, Loss: 5.063288715150621
Epoch: 3, Loss: 4.16600595580207
Epoch: 4, Loss: 3.5123263862397938
Epoch: 5, Loss: 2.956355579694112
Epoch: 6, Loss: 2.425340049796634
Epoch: 7, Loss: 1.937685982386271
Epoch: 8, Loss: 1.5158867438634236
Epoch: 9, Loss: 1.1650476853052776
Epoch: 10, Loss: 0.8869935575458738
Epoch: 11, Loss: 0.6788881907860438
Epoch: 12, Loss: 0.5269943124718136
Epoch: 13, Loss: 0.41521439055601755
Epoch: 14, Loss: 0.3340767681598663
Epoch: 15, Loss: 0.27245288723044925
Epoch: 16, Loss: 0.22396059764756096
Epoch: 17, Loss: 0.18826927741368613
Epoch: 18, Loss: 0.15911668886741
Epoch: 19, Loss: 0.13717951716648208
Epoch: 20, Loss: 0.11895610408650505


In [73]:
 def predict(model, question, threshold = 0.5):
   numerical_q = text_to_indices(question, vocab)
   question_tensor = torch.tensor(numerical_q).unsqueeze(0)
   output = model(question_tensor)
   probabilities = nn.functional.softmax(output, dim=1)
  #  print(probabilities)
   value, index = torch.max(probabilities, dim = 1)
   if(value < threshold):
     print("I don't know")
   print(list(vocab.keys())[index])


In [79]:
predict(model, "What is the largest planet in our solar system?")

jupiter
